In [ ]:
%pip install -r requirements.txt

In [ ]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire.utils import decode

In [ ]:
#Variables
website = 'https://jardinzoo.cl/gatos-166'
fileName = 'catsProducts'
folder = 'jardinzoo'

In [ ]:
chrome_options = Options()
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
chrome_options.add_argument('--auto-open-devtools-for-tabs')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable_encoding')

driver = webdriver.Chrome(options=chrome_options)

driver.execute_cdp_cmd("Network.enable", {})

driver.get(website)

wait = WebDriverWait(driver, 30)

In [ ]:
#Products
products = []

In [ ]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import random

def scan_products():
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article')))
    items = driver.find_elements(By.CSS_SELECTOR, 'article')
    
    for item in items:
        image_url = item.find_element(By.CSS_SELECTOR, 'img').get_attribute('src')
        product_url = item.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        title = item.find_element(By.CSS_SELECTOR, '.product-title').text
        price = item.find_element(By.CSS_SELECTOR, '.price').text
        products.append({
            'image_url': image_url,
            'product_url': product_url,
            'title': title,
            'price': price,
        })
    
def go_to_next_page():
    try:
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.next')))
        next_button.click()
        wait.until(EC.staleness_of(next_button))
        return True
    except NoSuchElementException:
        print("Botón 'next' no encontrado, final de la paginación.")
        return False
    except StaleElementReferenceException:
        print("El botón 'next' ya no es válido en el DOM.")
        return False
        
flag = True
while(flag):
    try:
        scan_products()
        time.sleep(random.uniform(3, 8))
        flag = go_to_next_page()
    except Exception as e:
        print(f"Error inesperado: {e}")
        flag = False
                

In [ ]:
print(products)
print(len(products))

In [ ]:
from selenium.common.exceptions import NoSuchElementException
def go_to_product(product):
    wait = WebDriverWait(driver, 120)
    driver.get(product['product_url'])
    
    try:
        product_tab_reference = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, "//a[contains(text(), 'Detalles del producto')]")
            )
        )
        product_tab_reference.click()
    except NoSuchElementException:
        print(f"No se encontró ean oculto en {product['title']}")
        
    try:
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.product-reference')))
        product_reference = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.product-reference')))
        product['ean'] = product_reference.find_element(By.CSS_SELECTOR, 'span').text
    except NoSuchElementException:
        print(f"La referencia no se encontró en el producto: {product['title']}")
        product['ean'] = ''

    try:
        product['brand'] = driver.find_element(By.CSS_SELECTOR, '.product-manufacturer').find_element(By.CSS_SELECTOR, 'a').text
    except NoSuchElementException:
        print(f"La marca no se encontró en el producto {product['title']}")
        product['brand'] = ''
    
    product['price'] = driver.find_element(By.CSS_SELECTOR, '.current-price').find_element(By.CSS_SELECTOR, 'span').get_attribute('content')
    product['currency'] = 'CLP'

for product in products:
    try:
        go_to_product(product)
    except Exception as e:
        print(f"Error inesperado producto {product['title']}: {e}")
    

In [ ]:
print(products)

In [ ]:
with open(f"{folder}/{fileName}.json", 'w', encoding='utf-8') as json_file:
    json.dump(products, json_file, ensure_ascii=False, indent=4)

print("Archivo JSON generado correctamente.")

In [ ]:
driver.quit()